In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

In [2]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2026-01-10 17:57:53.437 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\denni\.cache\mads_datasets\fashionmnist
2026-01-10 17:57:53.437 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\denni\.cache\mads_datasets\fashionmnist\fashionmnist.pt


In [3]:
accuracy = metrics.Accuracy()

In [4]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


2026-01-10 17:57:57.524 | INFO     | mltrainer.settings:check_path:60 - Created logdir c:\Users\denni\MADS\Portfolio-Dennis\1. Hypertuning gridsearch\modellogs


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2,num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

In [6]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

{'training': True}

In [7]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

In [8]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

2026-01-10 17:58:05.826 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20260110-175805
2026-01-10 17:58:09.011 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:00<00:00, 357.38it/s]
2026-01-10 17:58:09.702 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.9396 test 0.6915 metric ['0.7447']
100%|██████████| 100/100 [00:00<00:00, 304.98it/s]
2026-01-10 17:58:10.169 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.5757 test 0.5230 metric ['0.8102']
100%|██████████| 100/100 [00:00<00:00, 415.28it/s]
2026-01-10 17:58:10.540 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5153 test 0.5634 metric ['0.7900']
2026-01-10 17:58:10.541 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.5230, current loss 0.5634.Counter 1/10.
100%|██████████| 100/100 [00:00<00:00, 413.59it/s]
2026-01-10 17:58:10.931 | INFO     | mltrainer.traine

In [9]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        print(f"Units: {unit1}, {unit2}")

Units: 256, 256
Units: 256, 128
Units: 256, 64
Units: 128, 256
Units: 128, 128
Units: 128, 64
Units: 64, 256
Units: 64, 128
Units: 64, 64


In [ ]:
import torch

units = [256]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

for unit1 in units:
    for unit2 in units:
        model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.SGD,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()


2026-01-10 17:58:21.525 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20260110-175821
2026-01-10 17:58:21.527 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 937/937 [00:01<00:00, 481.75it/s]
2026-01-10 17:58:23.671 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.2541 test 2.1959 metric ['0.2956']
100%|██████████| 937/937 [00:01<00:00, 508.11it/s]
2026-01-10 17:58:25.717 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1039 test 1.9884 metric ['0.5247']
100%|██████████| 937/937 [00:01<00:00, 558.96it/s]
2026-01-10 17:58:27.581 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.8254 test 1.6574 metric ['0.5724']
100%|██████████| 937/937 [00:01<00:00, 531.21it/s]
2026-01-10 17:58:29.556 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.4956 test 1.3605 metric ['0.6089']
100%|██████████| 937/937 [00:01<00:00, 511.36it/s]
2026-0

: 